In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Pitching Airfoil 

In [62]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from time import time
from functools import partial
from jax import jit
import jax

# Dynamical Systems

In [3]:
from ICARUS.Systems.first_order_system import NonLinearSystem
from ICARUS.Systems.second_order_system import SecondOrderSystem
from ICARUS.Systems.integrate import BackwardEulerIntegrator, ForwardEulerIntegrator, RK4Integrator, RK45Integrator, CrankNicolsonIntegrator, GaussLegendreIntegrator, NewmarkIntegrator

In [4]:
def plot_results(x_data, t_data):
    # Plot the results
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    for key in list(x_data.keys()):
        # Compute cap so that we plot 1000 points at most
        cap = max(1, int(len(t_data[key]) / 1000))
        ax.plot(t_data[key][::cap], x_data[key][:,0][::cap], label=key)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Displacement (m)")
    ax.legend()


def test_all_integrators(system,x0, t0, t_end, dt0, compare_with_scipy = False):
    if isinstance(system, SecondOrderSystem):
        system2 = system.convert_to_first_order()
        newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)
    else:
        system2 = system
    
    integrator_rk4 = RK4Integrator(dt0, system2)
    integrator_feuler = ForwardEulerIntegrator(dt0, system2)
    integrator_beuler = BackwardEulerIntegrator(dt0, system2)
    integrator_rk45 = RK45Integrator(dt0, system2)
    integrator_gauss_legendre = GaussLegendreIntegrator(dt0, system2, n = 4)
    integrator_crank_nicolson = CrankNicolsonIntegrator(dt0, system2)

    integrators = [
        integrator_feuler,
        integrator_rk4,
        # integrator_rk45,
        integrator_beuler,
        # integrator_gauss_legendre,
        # integrator_crank_nicolson
     ]
    
    if isinstance(system, SecondOrderSystem):
        integrators.append(newmark)

    # Simulate the system using all the integrators
    x_data = {}
    t_data = {}


    for integrator in integrators:
        print(f"Simulating using {integrator.name} integrator")
        time_s = time()
        t_data[integrator.name], x_data[integrator.name] = integrator.simulate(x0, t0, t_end)
        time_e = time()
        print(f"\tSimulated using {integrator.name} integrator")
        print(f"\tTime taken: {time_e - time_s} seconds")

    if compare_with_scipy:
        print("Simulating using scipy RK45")
        time_s = time()
        sol = solve_ivp(system, [t0, t_end], x0, method='RK45', t_eval=np.linspace(0, t_end, 1000),  rtol=1e-6, atol=1e-6)
        t_data["scipy_rk45"] = sol.t
        x_data["scipy_rk45"] = sol.y.T
        time_e = time()
        print(f"\tSimulated using scipy RK45")
        print(f"\tTime taken: {time_e - time_s} seconds")

    plot_results(x_data, t_data)

# Simple Mass-Damper System

In [5]:
# Define a simple m-c-k system
m = 1.0
c = 0.1
k = 1.0


def f(t:float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        x[1],                            # x' = v
        -c /m * x[1] - k/m * x[0]        # v' = a = -c/m * v - k/m * x
    ])

# Create the system
system = NonLinearSystem(f)

# Test the integrators
test_all_integrators(system, jnp.array([1.0, 0.0]), 0.0, 100.0, 0.0001, compare_with_scipy = True)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 0.6831948757171631 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 0.38686180114746094 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 1.0899360179901123 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.32195186614990234 seconds


# Second Order Systems

In [6]:
# Define a 2nd order system 
m1 = 1.0
c1 = 0.1
k1 = 1.0

m2 = 1.0
c2 = 0.1
k2 = 1.0

def M(t,x):
    return jnp.array(
    [
        [m1, 0],
        [0, m2]
    ])
# M = jnp.array([m])
def C(t,x):
    return jnp.array(
    [
        [0.023, 1.024], #[c1, 0],
        [-0.364, 3.31]  #[0, c2]
    ])
# C = jnp.array([c])

def f_int(t,x):
    return jnp.array(
    [
        [1.97, 0.034],  #[k1, -k1],
        [0.034, 3.95]   #[-k1, k1 + k2]
    ])
# f_int = jnp.array([k])

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        0.078,# 0.0,
        10*0.466*jnp.sin(t)
    ])
# f_ext = lambda t, x: jnp.array([0.0])

system = SecondOrderSystem(M, C, f_int, f_ext)

In [7]:
# Test the integrators
test_all_integrators(system, jnp.array([0.0, 0.0, 0.0, 0.0]), 0.0, 100.0, 1e-4, compare_with_scipy = True)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 1.205092191696167 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 2.950021982192993 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 2.2151262760162354 seconds
Simulating using Newmark integrator
	Simulated using Newmark integrator
	Time taken: 0.7656283378601074 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.730494499206543 seconds
